In [ ]:
!pip install rdkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import rdkit as rd

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

In [ ]:
df_CB2 = pd.read_csv('CB2_initial_file.csv', delimiter=";")
df_CKR5 = pd.read_csv('CKR5_initial_file.csv', delimiter=";")
df_DRD1 = pd.read_csv('DRD1_initial_file.csv', delimiter=";")

names = ['cb2_mol_des.csv', 'ckr5_mol_des.csv', 'drd1_mol_des.csv']
datasets = [df_CB2, df_CKR5, df_DRD1]
new_datasets = []

counter = 0

for df in datasets:
  df_new = df[['Molecule ChEMBL ID', 'Smiles', 'Target Name', 'Standard Value' ]]
  df_new.dropna(how = 'any', inplace=True)
  df_new.reset_index(drop=True, inplace=True)
  df_new.rename(columns = {"Molecule ChEMBL ID" : "ID", "Standard Value" : "Ki"}, inplace=True)

  descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
  get_descriptors = rdMolDescriptors.Properties(descriptor_names)
  num_descriptors = len(descriptor_names)

  # Инициализация пустой матрицы для дескрипторов
  descriptors_set = np.empty((0, num_descriptors), float)

  # Замените DataFrame_CID на вашу переменную
  for _, row in df_new.iterrows():
    smiles = row['Smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
      descriptors = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
      descriptors_set = np.append(descriptors_set, descriptors, axis=0)

  # Создание DataFrame с дескрипторами
  df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)
  df_descriptors.reset_index(drop=True, inplace=True)

  # Объединение существующего датасета с новыми дескрипторами
  df_clean = pd.concat([df_new, df_descriptors], axis=1)

  df_clean.dropna(how = 'any', inplace=True)

  df_clean.to_csv(names[counter], index=False)

  new_datasets.append(df_clean)

  counter += 1

<ipython-input-10-421440d19a0f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dropna(how = 'any', inplace=True)
<ipython-input-10-421440d19a0f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.rename(columns = {"Molecule ChEMBL ID" : "ID", "Standard Value" : "Ki"}, inplace=True)
<ipython-input-10-421440d19a0f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dropna(how = 'any', inplace=True)
<ipython-input-10

In [ ]:
for i in new_datasets:
  i.head(10)
  i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948 entries, 0 to 5947
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               5948 non-null   object 
 1   Smiles                           5948 non-null   object 
 2   Target Name                      5948 non-null   object 
 3   Ki                               5948 non-null   float64
 4   exactmw                          5948 non-null   float64
 5   amw                              5948 non-null   float64
 6   lipinskiHBA                      5948 non-null   float64
 7   lipinskiHBD                      5948 non-null   float64
 8   NumRotatableBonds                5948 non-null   float64
 9   NumHBD                           5948 non-null   float64
 10  NumHBA                           5948 non-null   float64
 11  NumHeavyAtoms                    5948 non-null   float64
 12  NumAtoms            

In [ ]:
for df in new_datasets:
  print(df)


                 ID                                             Smiles  \
0      CHEMBL600801  CCCCCCC(C)(C)c1cc(C#N)c2c(c1)OC(C)(C)[C@@H]1CC...   
1       CHEMBL75571  COc1ccc2c(c1)c(CC(=O)N1CCOCC1)c(C)n2C(=O)c1ccc...   
2      CHEMBL201582  CC(C)c1nc2c(-c3ccc(Cl)cc3)n(-c3ccccc3Cl)nc2c(=...   
3      CHEMBL272944  Cc1c(C(=O)N2CCCCCC2)oc2ccc(S(=O)(=O)N3CC(C)CC(...   
4      CHEMBL404574  Cc1c(C(=O)NC2CCCCC2)oc2ccc(S(=O)(=O)N3CC(C)CC(...   
...             ...                                                ...   
5943  CHEMBL5178204  CCCCCn1cc(C(=O)N[C@@H](Cc2ccccc2)C(N)=O)c2cccnc21   
5944  CHEMBL5169682     CCCCn1nc(C(=O)N[C@H](C(N)=O)C(C)(C)C)c2ccccc21   
5945  CHEMBL5202843     CCCCn1cc(C(=O)N[C@H](C(N)=O)C(C)(C)C)c2ccccc21   
5946  CHEMBL5182619  COc1cccc2c(=O)c(C(=O)NC34CC5CC(CC(C5)C3)C4)cn(...   
5947  CHEMBL4753891     Cc1c(Br)cc(C(=O)NC2CCCCCC2)c(=O)n1Cc1ccc(F)cc1   

                   Target Name         Ki     exactmw         amw  \
0     Cannabinoid CB2 receptor      5.300 